In [2]:
from mnist import MNIST
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import operator

In [3]:
mndata = MNIST('C:\\Users\\tulsy\\Desktop\\CSE 253\\prog2')

In [4]:
images_train, labels_train = mndata.load_training()
images_test, labels_test = mndata.load_testing()

In [5]:
images_train = np.array(images_train)/127.5 -1
images_test = np.array(images_test)/127.5 -1
images_train = np.hstack((np.ones((60000,1)),images_train))
images_test = np.hstack((np.ones((10000,1)),images_test))

In [6]:
# Divide data into training and testing
Images_train =[]
lable_train =[]
Images_val =[]
Labels_val =[]

Images_train = images_train[:48000]
lable_train = labels_train[:48000]
Images_val = images_train[-12000:]
Labels_val =labels_train[-12000:]
print(np.shape(Images_train))


(48000, 785)


In [7]:
def one_hot_enc(data):
    sz = len (data)
    enc = np.zeros ((sz, 10))
    enc [range(sz), np.array(data)] = 1
    return enc

In [8]:
label_train_oh = one_hot_enc(lable_train)
label_val_oh = one_hot_enc(Labels_val)
label_test_oh = one_hot_enc(labels_test)

In [9]:
def Sigmoid(X, tan):
    if (tan  == 0):
        return 1.0/(1 + np.exp(-X))
    else:
        return 1.7159 * np.tanh((2/3)*X)

In [10]:
def SigmoidP(X, tan):
    if (tan == 0):
        return Sigmoid(X,tan)*(1-Sigmoid(X,tan))
    else:
        return 1.7159*(2/3)* (1- ((np.tanh((2/3)*X))*(np.tanh((2/3)*X))))

In [11]:
def get_softmax_prob (Input_O):
    ex = np.exp (Input_O)
    temp = np.sum(ex, axis=1)
    v = ex/np.resize(temp,(temp.shape[0],1))
    return v 

def get_softmax_pred (sm_prob):
    for i in range(len(sm_prob)):
        l = sm_prob[i]
        index, value = max(enumerate(l), key=operator.itemgetter(1))
        sm_prob[i] =0
        sm_prob[i][index] =1
    return sm_prob





In [12]:
#def Network(Training_Set_size, No_hidden_layer, No_hidden_nodes,lr1,lr2,epochs):
def feedforward(WIH, WHO,Images_train,batch_size, tan):
    N = batch_size
    Input_H = np.dot(Images_train,WIH.T)
    Activation_H = Sigmoid(Input_H, tan)
    Activation_H = np.hstack((np.ones((N,1)),Activation_H))
    Input_O = np.dot(Activation_H,WHO)
    Softmax_Input_O = get_softmax_prob(Input_O)
    return Input_H,Activation_H,Input_O,Softmax_Input_O

def backprop(Images_train, label_train_oh, Input_H, Activation_H, Input_O, Softmax_Input_O, WIH, WHO,l1,l2,batch_size,tan,momentum,M1,M2):
    N = batch_size
    Y_N = Softmax_Input_O
    diff = label_train_oh - Y_N
    gd_O = np.dot(diff.T,Activation_H)
    D_new=0
    U2_new=0
    if momentum == 1:
        D_old = M1
        Update = D_old*(0.9) - (l2 *(gd_O.T))/N
        WHO = WHO - Update
        D_new = Update
    else:
        WHO = WHO +(l2 *(gd_O.T))/N
    who_wb = WHO
    dot1 = np.dot(diff,who_wb.T)
    Ac_T = Activation_H.T
    Acativation_h_wb = Ac_T
    a_sigmoid = SigmoidP(Acativation_h_wb, tan)
    dot2 = np.multiply(dot1,a_sigmoid.T)
    dot3= np.dot(dot2.T,Images_train)
    if momentum == 1:
        D_old2 = M2
        UPDATE2 = D_old2*0.9 -(l1*dot3[1:])/N
        WIH = WIH - UPDATE2
        U2_new  = UPDATE2
    else:
        WIH = WIH +(l1*dot3[1:])/N
    return WIH,WHO,D_new,U2_new

def calOutputHidden(WIH, WHO,Images_val,tan):
    Input_H = np.dot(Images_val,WIH.T)
    Activation_H = Sigmoid(Input_H, tan)
    Activation_H = np.hstack((np.ones((len(Images_val),1)),Activation_H))
    Input_O = np.dot(Activation_H,WHO)
    Softmax_Input_O = get_softmax_prob(Input_O)
    Y_N = Softmax_Input_O
    return Y_N
    
def calculate_loss_and_acc (Y_N,Images_val,label_val_oh):
    err =0
    acc =[]
    acc_f =0
    for i in range(len(Images_val)):
        y_n = Y_N[i]
        t_n = label_val_oh[i]
        dot = np.dot(np.log(y_n),t_n)
        err -= dot
        idx1 = np.argmax(y_n)
        idx2 = np.argmax(t_n)
        if idx1 == idx2:
            acc.append(1)
        else:
            acc.append(0)
    acc_f = (1.0*sum(acc))/len(label_val_oh)
    return err/len(label_val_oh),acc_f
    
    

In [ ]:
def Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,H,l1,l2,epochs,shuffle1,momentum,tan,weight):
    H1 = H+1
    if weight == 1:
        WIH = np.random.normal(0, (1/np.sqrt(785)), (H,785))
        WHO = np.random.normal(0, (1/np.sqrt(785)), (H1,10))
    else:
        WIH = np.random.normal(0,0.00001, (H,785))
        WHO = np.random.normal(0,0.00001, (H1,10))
    batch_size = 128
    batch_no = int(len(Images_train)/128)
    acc_train =[]
    loss_train =[]
    acc_val =[]
    loss_val =[]
    acc_test =[]
    loss_test =[]
    epoch =[]
    M1=0
    M2=0
    for i in range(epochs):
        if shuffle1 == 1:
            print (i)
            Train_img_shuffle = []
            Train_label_shuffle = []
            shuffle = np.random.permutation(len(Images_train))
            #print(shuffle[:100])
            for sample in shuffle:
                Train_img_shuffle.append (Images_train[sample])
                Train_label_shuffle.append (label_train_oh[sample])
            #Images_train = Train_img_shuffle
            #label_train_oh = Train_label_shuffle
            #print(len(Images_train))
            #print(len(label_train_oh))
        for j in range(batch_no):
            start = j*128
            end = start+128
            if shuffle1 == 1:
                Images_T =Train_img_shuffle[start:end]
                Lables_T =Train_label_shuffle[start:end]
            else:
                Images_T =Images_train[start:end]
                Lables_T =label_train_oh[start:end] 
            Input_H,Activation_H,Input_O,Softmax_Input_O = feedforward(WIH,WHO,Images_T,batch_size,tan)
            WIH_n,WHO_n,M1_n,M2_n = backprop(Images_T, Lables_T, Input_H, Activation_H, Input_O, Softmax_Input_O, WIH, WHO,l1,l2,batch_size,tan,momentum,M1,M2)
            WIH = WIH_n
            WHO = WHO_n
            M1 = M1_n
            M2 = M2_n
        Y_N_train = calOutputHidden(WIH, WHO,Images_train,tan)
        err_t, acc_t = calculate_loss_and_acc(Y_N_train,Images_train,label_train_oh)
        acc_train.append(acc_t)
        loss_train.append(err_t)
        Y_N_val = calOutputHidden(WIH, WHO,Images_val,tan)
        err_v, acc_v = calculate_loss_and_acc(Y_N_val,Images_val,label_val_oh)
        
        acc_val.append(acc_v)
        loss_val.append(err_v)
        Y_N_test = calOutputHidden(WIH, WHO,images_test,tan)
        err_t1, acc_t1 = calculate_loss_and_acc(Y_N_test,images_test,label_test_oh)
        acc_test.append(acc_t1)
        loss_test.append(err_t1)
        epoch.append(i)
        #print(i)
    plt.plot(epoch,acc_train, color='blue', label ="Training Acc")
    plt.plot(epoch,acc_val, color ='green',label = "Validation Acc")
    plt.plot(epoch,acc_test, color ='orange', label = "Test Acc")
    plt.title("Accuracy v/s Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()
    plt.plot(epoch,loss_train, color='blue',label ="Training loss")
    plt.plot(epoch,loss_val, color ='green',label ="Validation loss")
    plt.plot(epoch,loss_test, color ='orange',label ="Test loss")
    plt.title("Loss v/s Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    print("Training Accuarcy",max(acc_train))
    print("Validation Accuarcy", max(acc_val))
    print("Test Accuracy", max(acc_test))
    print(np.argmax(np.array(acc_val)))



    
        

In [ ]:
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,64,0.1,0.1,500,0,0,0,0)


In [ ]:
# with shuffle
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,64,0.01,0.01,500,1,0,0,0)

In [ ]:
# tan
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,64,0.01,0.01,500,1,0,1,0)

In [ ]:
# weights
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,64,0.001,0.001,500,1,0,1,1)

In [ ]:
# moemtum
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,64,0.001,0.001,500,1,1,1,1)

In [ ]:
# With tan and shuffle data and hidden =32
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,32,0.01,0.01,500,1,1,1,1)
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,128,0.01,0.01,500,1,1,1,1)
Main(Images_train,label_train_oh,Images_val,label_val_oh,images_test,label_test_oh,500,0.01,0.01,500,1,1,1,1)